## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-14-50-42 +0000,bbc,Ukraine suffers heaviest attack as Trump criti...,https://www.bbc.com/news/articles/c2k19q4j07zo
1,2025-07-09-14-40-13 +0000,bbc,Lesotho declares state of disaster amid US tar...,https://www.bbc.com/news/articles/c4g8re5el87o
2,2025-07-09-14-33-23 +0000,bbc,Bin strike talks break down as council quits,https://www.bbc.com/news/articles/crk6rz4lg20o
3,2025-07-09-14-27-19 +0000,nypost,1-year-old killed by pet pit bull as mom tried...,https://nypost.com/2025/07/09/us-news/1-year-o...
4,2025-07-09-14-27-01 +0000,bbc,Palace await Euro fate as Lyon win relegation ...,https://www.bbc.com/sport/football/articles/cm...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,14
210,new,5
40,uk,5
101,two,4
420,tariffs,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
53,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,39
61,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,37
36,2025-07-09-10-00-00 +0000,nypost,"Trump, Elon Musk had ‘very troublesome ending,...",https://nypost.com/2025/07/09/us-news/trump-el...,35
18,2025-07-09-12-38-45 +0000,nypost,Trump boasted to donors that he threatened to ...,https://nypost.com/2025/07/09/us-news/trump-bo...,33
64,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,30


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
53,39,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
55,28,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
38,23,2025-07-09-10-00-00 +0000,nypost,White House chief of staff’s heart-wrenching l...,https://nypost.com/2025/07/09/us-news/white-ho...
24,21,2025-07-09-12-17-37 +0000,nypost,"Elon Musk’s AI chatbot Grok praises Hitler, sp...",https://nypost.com/2025/07/09/us-news/elon-mus...
61,19,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
3,18,2025-07-09-14-27-19 +0000,nypost,1-year-old killed by pet pit bull as mom tried...,https://nypost.com/2025/07/09/us-news/1-year-o...
8,18,2025-07-09-13-52-00 +0000,nypost,Kamala Harris’ interview on TikTok’s ‘Subway T...,https://nypost.com/2025/07/09/us-news/kamala-h...
13,17,2025-07-09-13-21-37 +0000,nypost,Georgia police officers break into car with tw...,https://nypost.com/2025/07/09/us-news/georgia-...
48,17,2025-07-09-06-54-08 +0000,nypost,Convicted murderer Derrick Groves eludes law e...,https://nypost.com/2025/07/09/us-news/convicte...
68,17,2025-07-08-17-38-56 +0000,bbc,France and UK to stop small boats together - M...,https://www.bbc.com/news/articles/cr4wdv69796o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
